In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from map_util import *

In [ ]:
def generate_map(height:int=64, width:int=64) -> tuple[np.ndarray, list]:
    # Map filled with walls
    map = np.ones((height, width)) * WALL

    def create_room(center_y, center_x, block_height, block_width):
        nonlocal map, height, width
        
        left = center_x - block_width // 2
        right = left + block_width
        top = center_y - block_height // 2
        bottom = top + block_height
        
        for y in range(top, bottom+1):
            for x in range(left, right+1):
                map[y, x] = EMPTY

    rooms = []

    max_size = 15

    while max_size > 4:
        bw = np.random.randint(3, max_size)
        bh = np.random.randint(3, max_size)

        map_choice = np.ones_like(map)

        for y in range(0, 1 + bh//2):
            for x in range(width):
                map_choice[y, x] = 0

        for y in range(height - 1 - (bh - bh//2), height):
            for x in range(width):
                map_choice[y, x] = 0

        for x in range(0, 1 + bw//2):
            for y in range(height):
                map_choice[y, x] = 0

        for x in range(width - 1 - (bw - bw//2), width):
            for y in range(height):
                map_choice[y, x] = 0

        for y in range(height):
            for x in range(0, width):
                if map[y,x] == EMPTY:
                    for cy in range(y - (bh-bh//2) - 1, y + bh//2 + 2):
                        for cx in range(x - (bw-bw//2) - 1, x + bw//2 + 2):
                            if 0 <= cy < height and 0 <= cx < width:
                                map_choice[cy, cx] = 0
                        
        I = np.argwhere(map_choice == 1)
        if len(I) == 0:
            max_size -= 1
            continue
        by,bx = I[np.random.choice(len(I))]

        create_room(by, bx, bh, bw)
        rooms.append((by, bx, bh, bw))

    rooms = np.array(rooms)

    todo = list(range(len(rooms)))

    # start with top left room
    current_index = np.argmin(np.array([math.sqrt(rooms[t][0]**2 + rooms[t][1]**2) for t in todo]))
    current = todo[current_index]

    while len(todo) > 1:
        current_y, current_x, _, _ = rooms[current]
        todo.remove(current)

        closest_indices = np.argsort([math.sqrt((rooms[t][0]-current_y)**2 + (rooms[t][1]-current_x)**2) for t in todo])[:2]
        next_index = np.random.choice(closest_indices)
        next = todo[next_index]

        # connect
        next_y, next_x, _, _ = rooms[next]

        if abs(next_x - current_x) > abs(next_y - current_y):
            if next_x != current_x:
                dir = np.sign(next_x - current_x)
                for x in range(current_x, next_x+dir, dir):
                    map[current_y, x] = EMPTY
                    map[current_y-dir, x] = EMPTY
            if next_y != current_y:
                dir = np.sign(next_y - current_y)
                for y in range(current_y, next_y+dir, dir):
                    map[y, next_x] = EMPTY
                    map[y, next_x-dir] = EMPTY
        else:
            if next_y != current_y:
                dir = np.sign(next_y - current_y)
                for y in range(current_y, next_y+dir, dir):
                    map[y, current_x] = EMPTY
                    map[y, current_x-dir] = EMPTY
            if next_x != current_x:
                dir = np.sign(next_x - current_x)
                for x in range(current_x, next_x+dir, dir):
                    map[next_y, x] = EMPTY
                    map[next_y-dir, x] = EMPTY

        current = next

    # place player in top left room
    top_left = np.argmin(np.array([math.sqrt(rx*rx + ry*ry) for ry,rx,_,_ in rooms]))
    ry, rx, rh, rw = rooms[top_left]
    py = ry
    px = rx
    map[py, px] = PLAYER

    # place exit in bottom right room
    bottom_right = np.argmax(np.array([math.sqrt(rx*rx + ry*ry) for ry,rx,_,_ in rooms]))
    ry, rx, rh, rw = rooms[bottom_right]
    ey = ry + (rh - rh//2)
    ex = rx + (rw - rw//2) + 1
    map[ey, ex] = EXIT
    
    return map, rooms

In [ ]:
def add_distances_from(distances:dict, position:tuple[int,int]):
    distances_extra,_ = compute_distances(map, position)

    distances_combined = {}
    for k in distances.keys():
        d = distances[k]
        if k in distances_extra:
            de = distances_extra[k]
            d += de
        distances_combined[k] = d
    return distances_combined

In [ ]:
def add_lock_around_exit():
    global map
    
    door_color = np.random.choice([DOOR_RED, DOOR_GREEN, DOOR_BLUE])
    
    exit_y, exit_x = exit_pos = np.argwhere(map == EXIT)[0]
    print(f'{exit_pos=}')
    
    if exit_y > 0 and map[exit_y-1, exit_x] == EMPTY:
        map[exit_y-1, exit_x] = door_color
    elif exit_y < map.shape[0]-1 and map[exit_y+1, exit_x] == EMPTY:
        map[exit_y+1, exit_x] = door_color
    elif exit_x > 0 and map[exit_y, exit_x-1] == EMPTY:
        map[exit_y, exit_x-1] = door_color
    elif exit_x < map.shape[1]-1 and map[exit_y, exit_x+1] == EMPTY:
        map[exit_y, exit_x+1] = door_color
        
    for y in range(exit_y-1, exit_y+2):
        for x in range(exit_x-1, exit_x+2):
            if 0 <= y < map.shape[0] and 0 <= x <map.shape[1] and map[y,x] == EMPTY:
                map[y,x] = WALL
        
    return door_color

In [ ]:
def get_room_distances(min_width:int, min_height:int) -> np.ndarray:
    global map, rooms
    distances = None
    positions = [tuple(pos) for pos in np.argwhere(np.logical_and(map != WALL, map != EMPTY))]
    for pos in positions:
        if distances is None:
            distances, _ = compute_distances(map, pos)
        else:
            distances = add_distances_from(distances, pos)
    for pos in positions:
        if pos in distances: del distances[pos]

    plot_distances(distances, map.shape[0], map.shape[1])

    def get_room_distance(room):
        nonlocal distances, min_height, min_width

        ry, rx, rh, rw = room
        if rh < min_height or rw < min_width: return np.nan
        
        pos = (ry,rx)
        return distances[pos] if pos in distances else np.nan

    room_distances = np.array([get_room_distance(r) for r in rooms])
    return room_distances

def get_farthest_room(min_height:int, min_width:int) -> tuple[int,int,int,int]:
    global map, rooms
    room_distances = get_room_distances(min_height, min_width)
    room_index = np.nanargmax(room_distances)
    return rooms[room_index]

def get_random_pos_in_room(room, margin_y:int, margin_x:int) -> tuple[int,int]:
    ry,rx,rh,rw = room
    
    min_y = ry - rh//2 + margin_y
    max_y = ry + (rh - rh//2) - margin_y
    min_x = rx - rw//2 + margin_x
    max_x = rx + (rw - rw//2) - margin_x
    
    positions = [(y,x) for x in range(min_x, max_x+1) for y in range(min_y, max_y+1) if map[y,x] == EMPTY]
    return positions[np.random.choice(len(positions))]
    
def add_key(key_color):
    global map
    
    room = get_farthest_room(5, 5)
    pos = get_random_pos_in_room(room, 1, 1)

    map[pos[0], pos[1]] = key_color

In [ ]:
door_to_key = {
    DOOR_RED: KEY_RED,
    DOOR_GREEN: KEY_GREEN,
    DOOR_BLUE: KEY_BLUE,
}

map, rooms = generate_map()
plot_map(map)
door_color = add_lock_around_exit()
plot_map(map)
key_color = door_to_key[door_color]
add_key(key_color)
plot_map(map)

In [ ]:
distances,_ = compute_distances(map, (py,px))
distances = add_distances_from(distances, (ey,ex))
del distances[(ey,ex)]
del distances[(py,px)]
plot_distances(distances, map.shape[0], map.shape[1])

(far_y,far_x),far_dist = sorted(distances.items(), key=lambda kvp: kvp[1])[-1]
map[far_y, far_x] = KEY_RED
plot_map(map)

distances = add_distances_from(distances, (far_y,far_x))
distances = add_distances_from(distances, (py,px))
plot_distances(distances, map.shape[0], map.shape[1])

(far_y,far_x),far_dist = sorted(distances.items(), key=lambda kvp: kvp[1])[-1]
map[far_y, far_x] = KEY_GREEN
plot_map(map)


In [ ]:
distances, paths = compute_distances(map, (py,px))
distances = add_distances_from(distances, (ey, ex))
del distances[(py,px)]
del distances[(ey,ex)]

def get_room_distance(room):
    ry, rx, rh, rw = room
    
    if rh < 5 or rw < 5: return 0
    
    pos = (ry,rx)
    return distances[pos] if pos in distances else 0

room_distances = [get_room_distance(r) for r in rooms]
room_index = np.argmax(room_distances)
room = rooms[room_index]


ry, rx, rh, rw = room

map[ry - 1, rx - 1] = WALL
map[ry - 1, rx] = WALL
map[ry - 1, rx + 1] = WALL
map[ry, rx - 1] = WALL
map[ry, rx] = KEY_BLUE
map[ry, rx + 1] = WALL
map[ry + 1, rx - 1] = WALL
map[ry + 1, rx] = DOOR_GREEN
map[ry + 1, rx + 1] = WALL

plot_map(map)